In [57]:
#import libraries
import pandas as pd

In [58]:
#import csv
worldBank = pd.read_csv('worldBank.csv')

#drop redundant columns
worldBank = worldBank.drop(columns='Indicator Code')
worldBank = worldBank.drop(columns='Country Code')

worldBank.head()

,Country Name,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,Unnamed: 60
0,Arab World,% of females ages 15-49 having comprehensive c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Arab World,% of males ages 15-49 having comprehensive cor...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arab World,"Adolescent fertility rate (births per 1,000 wo...",133.555013,134.159119,134.857912,134.504576,134.105211,133.569626,132.675635,131.665502,...,49.999851,49.887046,49.781207,49.672975,49.536047,49.383745,48.796558,48.196418,NaN,NaN
3,Arab World,Adults (ages 15+) and children (0-14 years) li...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arab World,Adults (ages 15+) and children (ages 0-14) new...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
#wide to long
worldBank = worldBank.melt(id_vars=['Country Name', 'Indicator Name'], var_name='Year', value_name='Value')
worldBank.head()

,Country Name,Indicator Name,Year,Value
0,Arab World,% of females ages 15-49 having comprehensive c...,1960,NaN
1,Arab World,% of males ages 15-49 having comprehensive cor...,1960,NaN
2,Arab World,"Adolescent fertility rate (births per 1,000 wo...",1960,133.555013
3,Arab World,Adults (ages 15+) and children (0-14 years) li...,1960,NaN
4,Arab World,Adults (ages 15+) and children (ages 0-14) new...,1960,NaN


In [60]:
#set index
worldBank = worldBank.set_index(['Country Name', 'Year'])

In [61]:
#multiindex pivot function
def multiindex_pivot(df, columns=None, values=None):
    #https://github.com/pandas-dev/pandas/issues/23955
    names = list(df.index.names)
    df = df.reset_index()
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index] # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df

#multiindex pivot
worldBank = multiindex_pivot(worldBank, 'Indicator Name', 'Value')
worldBank.head()

Indicator Name     % of females ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions)  \
Country Name Year                                                                                                                          
Afghanistan  1960                                                NaN                                                                       
             1961                                                NaN                                                                       
             1962                                                NaN                                                                       
             1963                                                NaN                                                                       
             1964                                                NaN                                                                       

Indicator Name     % of males ages 15-49 having comprehensive correct knowledge about HIV (2 prevent ways and reject 3 misconceptions)  \
Country Name Year                                                                                                                        
Afghanistan  1960                                                NaN                                                                     
             1961                                                NaN                                                                     
             1962                                                NaN                                                                     
             1963                                                NaN                                                                     
             1964                                                NaN                                                                     

Indicator Name     AIDS estimated deaths (UNAIDS estimates)  \
Country Name Year                                             
Afghanistan  1960                                       NaN   
             1961                                       NaN   
             1962                                       NaN   
             1963                                       NaN   
             1964                                       NaN   

Indicator Name     ARI treatment (% of children under 5 taken to a health provider)  \
Country Name Year                                                                     
Afghanistan  1960                                                NaN                  
             1961                                                NaN                  
             1962                                                NaN                  
             1963                                                NaN                  
             1964                                                NaN                  

Indicator Name     Adolescent fertility rate (births per 1,000 women ages 15-19)  \
Country Name Year                                                                  
Afghanistan  1960                                            145.321               
             1961                                            145.321               
             1962                                            145.321               
             1963                                            145.321               
             1964                                            145.321               

Indicator Name     Adults (ages 15+) and children (0-14 years) living with HIV  \
Country Name Year                                                                
Afghanistan  1960                                                NaN             
             1961                                                NaN             
             1962                                                NaN             
 